In [1]:
import numpy as np

### Решение для тестового контеста от Yandex


### Задача про интерполяцию полинома второй степени

Смысл задачи в том, чтобы по наборам исходных данных (x1,x2,x3,x4,x5 и y) количеством в 1000 шт., где y = f(x1,x2,x3,x4,x5) и является результатом рассчета неизвестного полинома 2й степени, получить способ определения 'y' из любого набора x1,x2,x3,x4,x5.

То есть, угадать этот полином.

Данное решение является аналитическим, а не численным, как градиентный спуск, ибо данные берутся не из жизни, а рассчитываются. Значит можно получить довольно точный результат.

Итак, мы расширяем набор, и превращаем его в прямое перечисление различных комбинаций вида:<br>
$$
x_1^{k1} * x_2^{k2} * x_3^{k3} * x_4^{k4} * x_5^{k5}
$$
где k1..5 может быть любым числом от 0 до 2.

В результате набор из 5 аргументов превращается в набор из 243 аргументов (3^5).

In [2]:
def transform_features(X, max_power=2):
    d = X.shape[1]

    f_powered = np.ones((X.shape), dtype='int64')
    for p in range(1, max_power+1):
        f_powered = np.column_stack((f_powered, X ** p))

    def recursion(factor, X_transformed, i=0):
        for j in range(i, i + d*(p + 1), d):
            product = factor * f_powered[:, j]
            if i < (d - 1):
                X_transformed = recursion(product, X_transformed, i + 1)
            else:
                X_transformed = np.column_stack((X_transformed, product))
        return X_transformed

    return recursion(f_powered[:, 0], np.ones((X.shape[0], 1), dtype='int')[:, 1:])

#### Класс линейной регрессии с аналитическим решением

Вообще, мне нравится идея, что если Xw = Y, то значит w = Y/X. Конечно, мы не можем поделить на матрицу. Кроме того, мы не можем найти обратную X^-1, потому что X не квадратная матрица, а 5x1000. 

Однако, в аналитическом решении используется псевдообратная матрица. 

$$
Xw = Y
$$
$$
(X_TX)w = X_TY
$$
$$
w = (X_TX)^{-1}X_TY
$$

Доказывать эту известную формулу тут не буду, надо только следить за тем, чтобы матрица X не содержала коллинеарных столбцов, потому что тогда произведение XT*X будет иметь определитель 0, а у такой матрицы не будет обратной. Но я не следил. И почему-то ничего страшного не случилось, хотя Х и Х^2 будут показывать высокий коэффициент коллинеарности Пирсона.

In [3]:
class linear_regression:
    def __init__(self):
        return None
    def fit(self, x, y):
        X = np.array(x).astype('float64')
        Y = np.array(y).astype('float64')
        self.weight = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
        return True
    def predict(self, x):
        X = np.array(x)
        Y = X.dot(self.weight)
        return Y
    def mean_squared_error(self, y_true, y_pred):
        mse = np.average((y_true - y_pred) ** 2, axis=0)
        return mse


In [4]:

X = []
X_test = []
Y = []
with open('input.txt', 'r') as f:
    for i in range(1000):
        in_str = f.readline().split('\t')
        X += [list(map(float, in_str))[:-1]]
        Y += list(map(float, in_str))[-1:]
    for i in range(1000):
        in_str = f.readline().split('\t')
        X_test += [list(map(float, in_str))]



X = np.array(X)
X_test = np.array(X_test)


trans_features_train = transform_features(X, max_power=2)
trans_features_test = transform_features(X_test, max_power=2)

target_train = Y

model = linear_regression()
model.fit(trans_features_train, target_train)

predict = model.predict(trans_features_test)


# разкомментить для вывода
# for target in predict:
# 	print(target)

Дальше идет часть, проверяющая результат, которую в Яндекс отправлять не нужно.

In [5]:
np.linalg.det(trans_features_train.T.dot(trans_features_train))

0.0

Ну вот, определитель равен 0.

In [6]:
target_test = []
with open('answers.txt', 'r') as f:
    for line in f:
        target_test += [float(line)]

In [7]:
print('mse\n',model.mean_squared_error(target_test, predict))

mse
 7.526150687119546e-09


А ошибка все равно низкая, несмотря на определитель.